In [2]:
import pandas as pd
from torchtext.data import Field, BucketIterator
import torch
import spacy
import en_core_web_sm
svoc = en_core_web_sm.load()
import numpy as np
from torchtext.data import TabularDataset

In [3]:
datao = pd.read_pickle("~/OneDrive/kph/processed.pkl")

In [45]:
# filter out less interesting keywords
datao['ext perc'].value_counts()

3     5403
4     4737
2     3903
5     2703
1     1883
6     1032
0      746
7      305
8      120
9       46
10      35
11      17
13       8
12       6
15       5
14       3
16       3
17       2
22       2
18       1
Name: ext perc, dtype: int64

In [46]:
# choose everything larger than 3 as the training and testing data set
datatrain = datao[datao['ext perc']>=3]
datatest = datao[datao['ext perc']<3]

In [47]:
datatrain

,SRC,TRG,keywords,ext perc
32460167,Clinical implications of serum neurofilament i...,O O O O O O O O O O O O O O O O O O O O O O B ...,"[Biomarker, Multiple sclerosis, Neurofilament ...",4
32460145,Effect of cognitive rehabilitation on neuropsy...,O O B I O O O O O O O O O O O B I O O O O O O ...,"[Clinical trial, Cognition, Cognitive rehabili...",4
32460086,B-cell depleting therapies may affect suscepti...,B I I I I O O O O O O O O O O B I O O O B O O ...,"[B-cell depleting therapies, COVID-19, DMTs, M...",4
32459513,The dichotomous role of the gut microbiome in ...,O O O O O B I O O O O B I O O O B I O O O O O ...,"[Alzheimer’s disease, Parkinson’s disease, gut...",5
32458728,Frailty in ageing persons with multiple sclero...,B O B O O B I O O O O B I O O O O O O O O O O ...,"[Fried’s phenotype, Multiple sclerosis, ageing...",5
...,...,...,...,...
7441643,Ghana--medical care amid economic problems. De...,B O O O O O O O O O O O O O O B B I O O O O B ...,"[Africa, Africa South Of The Sahara, Delivery ...",5
310887,Oral contraceptive therapy and systemic lupus ...,O O O O B I I O O O O O O O O B I I O O O O O ...,"[Biology, Case Studies, Contraception, Contrac...",3
197509,Estrogens for the menopause. Maximizing benefi...,O O O O O O O O O O O O O O O O O O O O O O O ...,"[Androgens--administraction and dosage, Biolog...",7
14312444,PORPHOBILINOGEN AND DELTA-AMINOLEVULINIC ACID ...,O O O O O O B O B I O O B O O O O O O O O O B ...,"[AMINO ACIDS, DEMYELINATION, DIAGNOSIS, DIFFER...",6


In [39]:
# prepare the train, test, validate data set
def prepare_csv(df_train, df_test,VAL_RATIO, seed=250):
    idx = np.arange(df_train.shape[0])
    np.random.seed(seed)
    np.random.shuffle(idx)
    val_size = int(len(idx) * VAL_RATIO)
    df_train.iloc[idx[val_size:], :].to_csv(
        "~/OneDrive/kph/ttd/train.csv", index=False)
    df_train.iloc[idx[:val_size], :].to_csv(
        "~/OneDrive/kph/ttd/val.csv", index=False)
    df_test.to_csv("~/OneDrive/kph/ttd/test.csv", index=False)

In [40]:
prepare_csv(datatrain.loc[:,['SRC','TRG']], datatest.loc[:,['SRC','TRG']],0.2, seed=250)

In [4]:
tokenizertrg = lambda x: x.split()
def tokenizersrc(text): # create a tokenizer function
    return [tok.text for tok in svoc.tokenizer(text)]

In [5]:
SRC = Field(sequential=True, tokenize=tokenizersrc)
TRG = Field(sequential=True, tokenize=tokenizertrg, use_vocab=False)

In [6]:
data_fields = [('SRC', SRC), ('TRG', TRG)]

In [7]:
train, val, test = TabularDataset.splits(
    path='~/OneDrive/kph/ttd/', train='train.csv',
    validation='val.csv', test='test.csv', format='csv',
    fields=data_fields,skip_header=True)


In [8]:
# first version with fixed embeddings
from torchtext import vocab

In [9]:
embvec = vocab.GloVe(name='840B', dim=300,cache='/home/pding/Documents/glove/')

In [10]:
SRC.build_vocab(train, vectors=embvec)

In [11]:

import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(250)

In [12]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [13]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embed_dim).from_pretrained(SRC.vocab.vectors)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [33]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 300
HIDDEN_DIM = 100

# Make up some training data
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]


In [18]:
tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

In [20]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

In [24]:
torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)

tensor([0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])

In [ ]:
#TEXT.vocab.set_vectors(vectors.stoi, vectors.vectors, vectors.dim)
#embedding = nn.Embedding(n_embed, embed_dim).from_pretrained(TEXT.vocab.vectors)

In [25]:
from torchtext.data import Iterator, BucketIterator

In [43]:
BATCH_SIZE = 3

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, val, test), 
    batch_size = BATCH_SIZE)
    #sort_key=lambda x: len(x.SRC))



In [44]:
a1 = next(iter(train_iterator))

ValueError: too many dimensions 'str'

In [30]:
vocab_size = len(SRC.vocab)

In [32]:
len(SRC.vocab.vectors)

72286